In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movies/data/disney_plus_titles.csv
/kaggle/input/movies/data/netflix_titles.csv
/kaggle/input/movies/data/hulu_titles.csv
/kaggle/input/movies/data/amazon_prime_titles.csv


In [43]:
data_netflix=pd.read_csv("/kaggle/input/movies/data/netflix_titles.csv")
data_disney=pd.read_csv("/kaggle/input/movies/data/disney_plus_titles.csv")
data_hulu=pd.read_csv("/kaggle/input/movies/data/hulu_titles.csv")
data_amazon=pd.read_csv("/kaggle/input/movies/data/amazon_prime_titles.csv")



In [44]:
data_netflix["platform"]="Netflix"
data_disney["platform"]="Disney"
data_amazon["platform"]="Amazon"
data_hulu["platform"]="Hulu"

In [45]:
df_all = pd.concat([data_netflix,data_disney,data_hulu,data_amazon],ignore_index=True)

In [46]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14738 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
 12  platform      22998 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.3+ MB


In [47]:
df_all["platform"].value_counts()

platform
Amazon     9668
Netflix    8807
Hulu       3073
Disney     1450
Name: count, dtype: int64

In [48]:
df_all.isnull().sum()

show_id             0
type                0
title               0
director         8260
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
platform            0
dtype: int64

In [49]:
df_all.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,platform
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix


In [50]:
fillnas=["director","cast","country","rating","date_added"]
for col in fillnas:
    df_all[col]=df_all[col].fillna("Unknown")


In [51]:
df_all["description"]=df_all["description"].fillna('')

In [52]:
df_all.dropna(subset=["release_year", "duration"], inplace=True)

In [53]:
df_all.info()  

<class 'pandas.core.frame.DataFrame'>
Index: 22516 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22516 non-null  object
 1   type          22516 non-null  object
 2   title         22516 non-null  object
 3   director      22516 non-null  object
 4   cast          22516 non-null  object
 5   country       22516 non-null  object
 6   date_added    22516 non-null  object
 7   release_year  22516 non-null  int64 
 8   rating        22516 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22516 non-null  object
 11  description   22516 non-null  object
 12  platform      22516 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.4+ MB


In [54]:
def clean_text(text):
    if isinstance(text,str):
        return text.lower().strip().replace(" ","")
    else:
        return ''

In [55]:
features=["director","cast","country","listed_in"]
for feature in features:
    df_all[feature]=df_all[feature].apply(clean_text)

In [56]:
def get_top3_cast(text):
    if isinstance(text, str):
        names = text.split(',')[:3]
        return ' '.join(names)
    return ''

In [57]:
def create_soup(x):
    director = (str(x['director']) + ' ') * 3
    genre = (str(x['listed_in']) + ' ') * 2
    cast = get_top3_cast(str(x['cast']))
    return x['title'] + ' ' + director + ' ' + cast + ' ' + genre + ' ' + x['description']

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
df_all["soup"]=df_all.apply(create_soup,axis=1)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_all['soup'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
df_all = df_all.reset_index(drop=True)

In [59]:
df_all["soup"].head().values

array(['Dick Johnson Is Dead kirstenjohnson kirstenjohnson kirstenjohnson  unknown documentaries documentaries  As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.',
       'Blood & Water unknown unknown unknown  amaqamata khosingema gailmabalane internationaltvshows,tvdramas,tvmysteries internationaltvshows,tvdramas,tvmysteries  After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.',
       'Ganglands julienleclercq julienleclercq julienleclercq  samibouajila tracygotoas samueljouy crimetvshows,internationaltvshows,tvaction&adventure crimetvshows,internationaltvshows,tvaction&adventure  To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.',
       'Jailbirds New Orleans unknown unknown unknown  unknow

In [60]:
print(tfidf_matrix.shape)

(22516, 81692)


In [61]:
indices = pd.Series(df_all.index,index=df_all["title"]).drop_duplicates()

def get_recommendations(title,cosine_sim=cosine_sim):
    try:
        idx=indices[title]
    except KeyError:
        return "film bulunamadı ismi kontrol et"
    sim_scores=list(enumerate(cosine_sim[idx]))    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df_all[['title', 'platform', 'description']].iloc[movie_indices]
        

In [62]:
print(get_recommendations('Squid Game'))

                 title platform  \
1554  Children of Adam  Netflix   
3684         Kakegurui  Netflix   
7590             Nobel  Netflix   
1373              50M2  Netflix   
331      Darwin’s Game  Netflix   
3534           The Spy  Netflix   
8746        Winter Sun  Netflix   
3288    Chief of Staff  Netflix   
1059    House of Cards  Netflix   
6143  American Odyssey  Netflix   

                                            description  
1554  Amid crime and corruption, the lives of two co...  
3684  Yumeko Jabami enrolls at Hyakkaou Private Acad...  
7590  A series of incidents in Afghanistan set compl...  
1373  Seeking to uncover the truth about his past, a...  
331   High schooler Kaname activates a mysterious mo...  
3534  In the 1960s, Israeli clerk-turned-secret agen...  
8746  Years after ruthless businessmen kill his fath...  
3288  As a chief of staff in the National Assembly, ...  
1059  A ruthless politician will stop at nothing to ...  
6143  An elite soldier, a corporat

In [63]:
# Inception için test edelim
print(get_recommendations('Inception'))

                  title platform  \
3798   Rim of the World  Netflix   
946            Stargate  Netflix   
8039     Small Soldiers  Netflix   
7451   Midnight Special  Netflix   
8528   The Time Machine  Netflix   
6177          Apollo 18  Netflix   
7578               Next  Netflix   
8665     V for Vendetta  Netflix   
610          Underworld  Netflix   
3816  See You Yesterday  Netflix   

                                            description  
3798  Stranded at a summer camp when aliens attack t...  
946   An Egyptologist joins a mission into the unkno...  
8039  When the Commando Elite, a group of toy action...  
7451  When his son exhibits supernatural powers, a f...  
8528  Based on the H.G. Wells classic, this adventur...  
6177  This sci-fi thriller suggests that Apollo 17, ...  
7578  Las Vegas magician Cris can see a few minutes ...  
8665  A masked freedom fighter known as "V" takes up...  
610   A vampire warrior begins to question her missi...  
3816  As two teen prodi

In [64]:
import joblib
joblib.dump(df_all,"movie_data.pkl")
joblib.dump(cosine_sim,"similarity_matrix.pkl")


['similarity_matrix.pkl']